In [ ]:
from google.colab import data_table, drive
import os

data_table.enable_dataframe_formatter()
drive.mount('/content/drive/')
data_path = "/content/drive/MyDrive/Colab Notebooks/DSC 245 - Intro to Causal Inference"

Mounted at /content/drive/


In [43]:
# del train_loader, test_loader, model
# import gc
# torch.cuda.empty_cache()
# gc.collect()

0

## This notebook uses GCN and GAT to estimate average treatment effect based on BN Connectivity and Causally Weighted Feature Embeddings

In [ ]:
!pip install causalml
!pip install torch-scatter
!pip install torch-sparse
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
 

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from gensim.models import word2vec
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm
random.seed(2022)
np.random.seed(2022)

In [ ]:
# !pip install --upgrade gensim
# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df_train=pd.read_csv("criteo_sampled/criteo_train.csv",index_col=0)
df_test=pd.read_csv("criteo_sampled/criteo_test.csv",index_col=0)

In [ ]:
# Shuffle the DataFrame
multi_data=pd.read_csv(os.path.join(data_path,"criteo-uplift-v2.1.csv"))
multi_data.columns=['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'T', 'y', 'visit', 'tau']
df_shuffled = multi_data.sample(frac=1, random_state=2022).reset_index(drop=True)

# Calculate the number of samples for train_val set
train_val_size = int(len(df_shuffled) * 0.2)
train_val_set = df_shuffled.iloc[:train_val_size]

# Calculate the number of samples for the training set within the train_val set
train_size_within_train_val = int(train_val_size * 0.8)

# Split train_val set into train and validation sets
df_train = train_val_set.iloc[:train_size_within_train_val]
df_test = train_val_set.iloc[train_size_within_train_val:]

In [ ]:
# load causal weighting
ate_list=[]
ate=pd.read_excel(os.path.join(data_path,"feats_ate_x13.xlsx"),index_col=0)
for i in [c for c in df_train.columns[:-4]]+["visit","treatment"]:
    ate_list.append(float(ate[ate['Feature']==i]["ATE"].values))
ate_list

In [63]:
# load edge index
edge_index=pd.read_csv(os.path.join(data_path,'criteo_edge_index_ges_14.csv'))
edge_index=torch.from_numpy(np.transpose(np.array(edge_index)))
edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,
          5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,
          8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 11, 11,
         12, 12, 12, 12, 12, 12, 12, 12],
        [ 1,  2,  3,  6,  8,  9, 10, 11,  2,  3,  7,  8, 11,  3,  4,  8, 10, 11,
          1,  4,  6,  7,  8,  9, 10, 12,  2,  3,  7,  8, 10, 11,  2,  3,  4,  8,
          4, 10, 11,  0,  1,  2,  3,  4,  5,  6,  8, 10, 11, 12,  4, 11,  3,  4,
          1,  4,  5,  6,  8,  9, 10, 11]])

In [64]:
# load node embedding
model_dw=word2vec.Word2Vec.load(os.path.join(data_path,"deepwalk_10d_x13.model"))
model_n2v=word2vec.Word2Vec.load(os.path.join(data_path,"Node2Vec_10d_x13.model"))

lst_dw=[]
lst_n2v=[]
for i in range(14):
    lst_dw.append(model_dw.wv[i])
    lst_n2v.append(model_n2v.wv[i])
len(lst_dw), len(lst_n2v)

(14, 14)

In [65]:
from scipy.stats import entropy
def R2(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    R2 = 1 - np.sum(np.square(y_actual-y_predicted)) / np.sum(np.square(y_actual-np.mean(y_actual)))
    return R2

def MAE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAE = np.mean(abs(y_actual-y_predicted))
    return MAE

def RMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    return RMSE

def CVRMSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    RMSE = np.sqrt(np.mean(np.square(y_actual-y_predicted)))
    CVRMSE = RMSE/np.mean(y_actual)
    return CVRMSE

def MAPE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MAPE=np.mean(abs((y_actual-y_predicted)/y_actual))
    return MAPE

def MSE(y_predicted, y_actual):
    y_predicted = np.asarray(y_predicted, dtype=float)
    y_actual = np.asarray(y_actual, dtype=float)

    MSE = np.mean(abs(y_actual-y_predicted)**2)
    return MSE

def kl_divergence(p, q):

    p = np.asarray(p, dtype=float)
    q = np.asarray(q, dtype=float)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

def kl_divergence(y_predicted, y_actual):

    stacked_values = np.hstack((y_predicted, y_actual))
    stacked_low = np.percentile(stacked_values, 0.1)
    stacked_high = np.percentile(stacked_values, 99.9)
    bins = np.linspace(stacked_low, stacked_high, 100)

    distr = np.histogram(y_predicted, bins=bins)[0]
    distr = np.clip(distr / distr.sum(), 0.001, 0.999)
    true_distr = np.histogram( y_actual, bins=bins)[0]
    true_distr = np.clip(true_distr / true_distr.sum(), 0.001, 0.999)

    kl = entropy(distr, true_distr)
    return kl

In [66]:
import torch
from torch_geometric.data import InMemoryDataset, download_url


class Mytrain(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y


    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_train.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_train[i]
            if feats_mode =='causal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

                causal_weighted= np.multiply(np.array(X),np.array(ate_list))
                weighted_feats.append(causal_weighted)

                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(causal_weighted) # feature (10d), causally weighted features
                                                        )))
            elif feats_mode =='equal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

                weighted= np.array(X)
                weighted_feats.append(weighted)

                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(weighted) # feature (10d), equally weighted features
                                                        )))
            else:
                t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding


                for j in range(14):
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        )))


            Y = y_train[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)

            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])


class Mytest(InMemoryDataset):
    def __init__(self, root,transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
       # ,self.edge_index,self.y=x,edge_index,y


    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list=[]
        weighted_feats=[]
        for i in range(x_test.shape[0]):
            Edge_index = edge_index.type(torch.long)
            X =x_test[i]

            if feats_mode =='causal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

                causal_weighted= np.multiply(np.array(X),np.array(ate_list))
                weighted_feats.append(causal_weighted)

                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(causal_weighted) # feature (10d), causally weighted features
                                                        )))
            elif feats_mode =='equal':
                t=torch.zeros(14,25) # 25 dimensions = 1+10+14 = node number, node embedding, features

                weighted= np.array(X)
                weighted_feats.append(weighted)

                for j in range(14): # for j-th feature
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        list(weighted) # feature (10d), equally weighted features
                                                        )))
            else:
                t=torch.zeros(14,11) # 11 dimensions = 1+10 = node number, node embedding


                for j in range(14):
                  t[j]=torch.from_numpy(np.concatenate(( [float(X[int(j)])], # node (1d)
                                                        list(lst_dw[int(j)]), # node embeddings (10d), prior embeddings from DeepWalk/Node2Vec
                                                        )))

            Y = y_test[i].reshape(-1,1).to(torch.float32)
            data = Data(x=t, edge_index=Edge_index, y=Y)

            data_list.append(data)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [67]:
feats_mode='causal'
# feats_mode ='equal'
#feats_mode ='noweighting'

col=df_train.columns
y_train=torch.from_numpy(np.array(df_train['y'])).reshape(df_train.shape[0],1).to(torch.float32)
x_train=torch.from_numpy(np.array(df_train[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
Mydata_train=Mytrain(".\mydata_sampled_ate\MYdata_train")

y_test=torch.from_numpy(np.array(df_test['y'])).reshape(df_test.shape[0],1).to(torch.float32)
x_test=torch.from_numpy(np.array(df_test[[i for i in col[:-4]]+["visit","T"]])).to(torch.float32)
Mydata_test=Mytest(".\mydata_sampled_ate\MYdata_test")

# GCN

In [12]:
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in tqdm(train_loader):
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()


    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in tqdm(test_loader):
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()


    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl

In [13]:
import torch
from torch_scatter import scatter_add
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(25, 64)
        self.conv2 = GCNConv(64, 10)
        # self-attention layer

        self.f1 = torch.nn.Linear(140,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        # x = self.attention
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = x.reshape(-1,140)
        x = self.f1(x)
        x = self.f2(x)
        return x

In [14]:
num_epochs = 32
batch_size = 1024*128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
crit = F.mse_loss
train_loader = DataLoader(Mydata_train, batch_size=batch_size, num_workers=8)
test_loader = DataLoader(Mydata_test, batch_size=batch_size, num_workers=8)

for epoch in tqdm(range(num_epochs)):
    loss,r2,mse,kl=train()
    if epoch %5==0:
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

y_predicted = []
for data in test_loader:
    loss = 0
    data = data.to(device)
    output = model(data)
    label = data.y.to(device)
    y_predicted +=(output).cpu().detach().ravel().tolist()
df_test["y_hat"]=y_predicted

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

R2:-36.427480  MSE:0.110811  KL:2.918435  MAE:0.242970  RMSE:0.332883  CVRMSE:112.100421  train_loss:
0.11081117324426672


  0%|          | 0/5 [00:00<?, ?it/s]

R2:-1.385924  MSE:0.006896  KL:5.294770  MAE:0.068603  RMSE:0.083045  CVRMSE:28.647304  test_loss:
6.193505724034859e-08


...

  0%|          | 0/5 [00:00<?, ?it/s]

R2:-0.000065  MSE:0.002891  KL:0.000762  MAE:0.006762  RMSE:0.053765  CVRMSE:18.546823  test_loss:
2.6176862749540374e-08


  0%|          | 0/18 [00:00<?, ?it/s]

R2:-0.012304  MSE:0.002997  KL:0.636578  MAE:0.008229  RMSE:0.054746  CVRMSE:18.436034  

<ipython-input-14-15a65c7598eb>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["y_hat"]=y_predicted


In [15]:
len(y_predicted)

559184

In [ ]:
train_loader = DataLoader(Mydata_train, batch_size=batch_size)
for epoch in range(num_epochs):
    loss,r2,mse,kl=train()
    if epoch %5==0:
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

In [16]:
from causalml.metrics import *

# Assuming 'uplift' is your DataFrame that already includes the necessary columns
uplift=df_test.copy()
auuc = auuc_score(uplift, outcome_col='y', treatment_col='T', prediction_cols=['y_hat'])
gcn_auuc = pd.DataFrame(auuc[["y_hat","Random"]], columns=['auuc'])
print(gcn_auuc)

            auuc
y_hat   0.549209
Random  0.499138


In [ ]:
# estimate area under the uplift curve (AUUC)
from causalml.metrics import *
uplift=df_test.copy()
tau_hat=pd.concat([t0,t1], axis=0, join="inner")
uplift=pd.concat([uplift,tau_hat], axis=1, join="inner")
uplift = uplift.loc[:,~uplift.columns.duplicated()]

auuc=auuc_score(uplift, outcome_col='y', treatment_col='T')
gcn_auuc=pd.DataFrame(auuc[["y_hat","Random"]],columns=['auuc'])
gcn_auuc

NameError: name 't0' is not defined

# GAT

In [68]:
from uuid import RFC_4122
import torch
import math
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GATConv

from torch_geometric.utils import add_self_loops,degree
from torch_geometric.datasets import Planetoid
import ssl
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # self.gat1=GATConv(in_channels=25,out_channels=8,heads=8,dropout=0.6)
        # self.gat2=GATConv(in_channels=64,out_channels=10,heads=1,dropout=0.6)
        self.gat1=GATConv(in_channels=25,out_channels=8,heads=8)
        self.gat2=GATConv(in_channels=64,out_channels=10,heads=1)
        self.f1 = torch.nn.Linear(140,32)
        self.f2 = torch.nn.Linear(32,1)
    def forward(self,data):
        x,edge_index=data.x, data.edge_index
        x=self.gat1(x,edge_index)
        x=self.gat2(x,edge_index)
        x=x.reshape(-1,140)
        x = self.f1(x)
        x = self.f2(x)
        return x

ssl._create_default_https_context = ssl._create_unverified_context
def train():
    model.train()
    loss_all = 0
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in tqdm(train_loader):
        loss = 0
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
        y_actual +=(label).cpu().detach().ravel().tolist()
        y_predicted +=(output).cpu().detach().ravel().tolist()


    loss=loss_all/len(Mydata_train)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2,mse,kl


def val():
    model.eval()
    y_actual = []
    y_predicted = []
    loss_all=0
    for data in test_loader:
      loss = 0
      data = data.to(device)
      output = model(data)
      label = data.y.to(device)
      y_actual +=(label).cpu().detach().ravel().tolist()
      y_predicted +=(output).cpu().detach().ravel().tolist()
      loss = crit(output, label)
      loss_all += loss.item()


    loss = loss_all / len(Mydata_test)
    r2=R2(y_predicted, y_actual)
    mse = MSE(y_predicted, y_actual)
    kl=kl_divergence(y_predicted, y_actual)

    print("R2:%f" % (R2(y_predicted, y_actual)),end='  ')
    print("MSE:%f" % (MSE(y_predicted, y_actual)),end='  ')
    print("KL:%f" % (kl_divergence(y_predicted, y_actual)),end='  ')
    print("MAE:%f" % (MAE(y_predicted, y_actual)),end='  ')
    print("RMSE:%f" % (RMSE(y_predicted, y_actual)),end='  ')
    print("CVRMSE:%f" % (CVRMSE(y_predicted, y_actual)),end='  ')

    return loss,r2, mse,kl



num_epochs = 32
batch_size = 1024*64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
crit = F.mse_loss
train_loader = DataLoader(Mydata_train, batch_size=batch_size, num_workers=8)
test_loader = DataLoader(Mydata_test, batch_size=batch_size, num_workers=8)

for epoch in tqdm(range(num_epochs)):
    loss,r2,mse,kl=train()
    if epoch %5==0:
        print('train_loss:')
        print(loss)

        loss,r2,mse,kl=val()
        print('test_loss:')
        print(loss)

y_predicted = []
for data in test_loader:
    loss = 0
    data = data.to(device)
    output = model(data)
    label = data.y.to(device)
    y_predicted +=(output).cpu().detach().ravel().tolist()
df_test["y_hat"]=y_predicted

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

R2:-153.950420  MSE:0.458760  KL:3.014438  MAE:0.451585  RMSE:0.677318  CVRMSE:228.091096  train_loss:
0.45876018705785443
R2:-0.676508  MSE:0.004846  KL:3.071976  MAE:0.028043  RMSE:0.069612  CVRMSE:24.013657  test_loss:
7.827114245222147e-08


  0%|          | 0/35 [00:00<?, ?it/s]

R2:-1.388075  MSE:0.007070  KL:3.215106  MAE:0.052932  RMSE:0.084085  CVRMSE:28.316266  

...

  0%|          | 0/35 [00:00<?, ?it/s]

R2:0.084880  MSE:0.002709  KL:0.521889  MAE:0.008520  RMSE:0.052052  CVRMSE:17.528755  train_loss:
0.002709388099476158
R2:0.087619  MSE:0.002637  KL:0.559879  MAE:0.008446  RMSE:0.051354  CVRMSE:17.715100  test_loss:
4.264347134194523e-08


  0%|          | 0/35 [00:00<?, ?it/s]

R2:0.086501  MSE:0.002705  KL:0.585043  MAE:0.008483  RMSE:0.052006  CVRMSE:17.513215  

<ipython-input-68-cb51de2e0e84>:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["y_hat"]=y_predicted


In [ ]:
# estimate area under the uplift curve (AUUC)
from causalml.metrics import *
uplift=df_test.copy()
tau_hat=pd.concat([t0,t1], axis=0, join="inner")
uplift=pd.concat([uplift,tau_hat], axis=1, join="inner")
uplift = uplift.loc[:,~uplift.columns.duplicated()]

auuc=auuc_score(uplift, outcome_col='y', treatment_col='T', treatment_effect_col='tau')
gat_auuc=pd.DataFrame(auuc[["y_hat","Random"]],columns=['auuc'])
gat_auuc

,auuc
y_hat,0.880700
Random,0.499185


In [69]:
from causalml.metrics import *

# Assuming 'uplift' is your DataFrame that already includes the necessary columns
uplift=df_test.copy()
auuc = auuc_score(uplift, outcome_col='y', treatment_col='T', prediction_cols=['y_hat'])
gcn_auuc = pd.DataFrame(auuc[["y_hat","Random"]], columns=['auuc'])
print(gcn_auuc)

            auuc
y_hat   0.657535
Random  0.499138


In [ ]:
result=pd.DataFrame(columns=[['AUUC']])
result.loc['S Learner(LR)','AUUC']=0.497983
result.loc['S Learner(XGB)','AUUC']=0.875572
result.loc['S Learner(LGBM)','AUUC']=0.883033

result.loc['GCN (Struct)','AUUC']=0.501865
result.loc['GCN (Struct+Feature)','AUUC']=0.721959
result.loc['GCN (Struct+Causal Weighting)','AUUC']=gcn_auuc.loc["y_hat"].values

result.loc['GAT (Struct)','AUUC']=0.544286
result.loc['GAT (Struct+Feature)','AUUC']=0.847630
result.loc['GAT (Struct+Causal Weighting)','AUUC']=gat_auuc.loc["y_hat"].values

result

,AUUC
S Learner(LR),0.497983
S Learner(XGB),0.875572
S Learner(LGBM),0.883033
GCN (Struct),0.501865
GCN (Struct+Feature),0.721959
GCN (Struct+Causal Weighting),0.732616
GAT (Struct),0.544286
GAT (Struct+Feature),0.84763
GAT (Struct+Causal Weighting),0.8807


In [ ]:
result=pd.DataFrame(columns=[['GCN','GAT']])
result.loc['equal weighting','GCN']=0.721959
result.loc['equal weighting','GAT']=0.847630

result.loc['causal weighting','GCN']=gcn_auuc.loc["y_hat"].values
result.loc['causal weighting','GAT']=gat_auuc.loc["y_hat"].values
result

,GCN,GAT
equal weighting,0.721959,0.84763
causal weighting,0.732616,0.8807


In [ ]:
result=pd.DataFrame(columns=[['GCN','GAT']])
result.loc['Hill Climbing (equal weighting)','GCN']=0.721959
result.loc['Hill Climbing (equal weighting)','GAT']=0.794647 # 0.684647
result.loc['Hill Climbing (causal weighting)','GCN']=0.733972
result.loc['Hill Climbing (causal weighting)','GAT']=0.836097 # 0.727444

result.loc['Noise (equal weighting)','GCN']=0
result.loc['Noise (equal weighting)','GAT']=0
result.loc['Noise (causal weighting)','GCN']=0.824441
result.loc['Noise (causal weighting)','GAT']=0.549209

result.loc['GES (equal weighting)','GCN']=0
result.loc['GES (equal weighting)','GAT']=0
result.loc['GES (causal weighting)','GCN']=0.743254
result.loc['GES (causal weighting)','GAT']=0.816024    # 0.717556
result